## MVP for GrowBikeNet implementation

In [ ]:
# import libraries
import os
import cv2
import glob
import re
import pathlib

import osmnx as ox
import pandas as pd
import geopandas as gpd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt

import shapely
from shapely.prepared import prep
from shapely.geometry import Point, LineString, MultiLineString

from itertools import combinations

### User Input:

In [ ]:
city_name = 'Oelde'
seed_point_spacing = 3000 # distance between seed points, in meters
orig_crs = '4326'
proj_crs = '3857'
seed_point_delta = 500 # maximal distance between seed point and actual point in OSM data, in meters

### Data from OSM

In [ ]:
# fetch street network data from osmnx
g = ox.graph_from_place(
    city_name, network_type='all'
)
g_undir = g.to_undirected().copy() # convert to undirected (dropping OSMnx keys!)

# # plausibility check
# ox.plot_graph(g, figsize=(10, 10))

### Street network data

In [ ]:
# export osmnx data to gdfs
nodes, edges = ox.graph_to_gdfs(
    g_undir,
    nodes=True,
    edges=True,
    node_geometry=True,
    fill_edge_geometry=True
)

# save "original" graph data (in orig_crs)
nodes.to_file("nodes.gpkg", driver='GPKG')
edges.to_file("edges.gpkg", driver='GPKG')

# replace after dropping edges with key = 1
edges = edges.loc[:,:,0].copy()
# this also means we are dropping the "key" level from edge index (u,v,key becomes: u,v)

# project geometries of nodes and edges
edges = edges.to_crs(proj_crs)
nodes = nodes.to_crs(proj_crs)

# add osm ID as column to node gdf
nodes["osmid"] = nodes.index

### Seed points

In [ ]:
# get convex hull around edge area
hull = edges.union_all().convex_hull
# get bounds of hull
latmin, lonmin, latmax, lonmax = hull.bounds

# https://stackoverflow.com/questions/66010964/fastest-way-to-produce-a-grid-of-points-that-fall-within-a-polygon-or-shape
# populate hull bbox with evenly spaced seeding points
points = []
for lat in np.arange(latmin, latmax, seed_point_spacing):
    for lon in np.arange(lonmin, lonmax, seed_point_spacing):
        points.append(Point((round(lat,4), round(lon,4))))

# keep only those seed points that are within the hull polygon
prep_polygon = prep(hull)
valid_points = []
valid_points.extend(filter(prep_polygon.contains, points))

# store seed points in gdf
seed_points = gpd.GeoDataFrame(
    {"geometry": valid_points},
    crs = proj_crs
)

# # plausibility check 
# seed_points.plot() 

### Snap Seed points to OSM nodes

In [ ]:
# query nearest OSM nodes with sindex
q = nodes.sindex.nearest(seed_points.geometry)
seed_points["osmid"] = None
seed_points.iloc[q[0], -1] = list(nodes.iloc[q[1]]["osmid"])

# create a subset of OSM nodes - only those that seed points are snapped to
nodes_subset = nodes.loc[
    nodes.osmid.isin(seed_points.osmid)
].copy().reset_index(drop=True)

# merge seed points gdf (gives us the generated seed point location, "geometry_generated")
# with nodes subset gdf (gives us all other columns)
# (we need the geometry_generated column only for filtering by distance)
seed_points_snapped = pd.merge(
    left=seed_points, 
    right=nodes_subset,
    how="inner",
    on="osmid",
    suffixes=["_generated", "_osm"]
)

# define our boolean distance_condition filter:
# snapped seed points must be not more than seed_point_delta away
# from their OSM nodes
distance_condition = seed_points_snapped.geometry_generated.distance(
    seed_points_snapped.geometry_osm) <= seed_point_delta

# filter seed_points_snapped df by distance condition
seed_points_snapped = seed_points_snapped[distance_condition].reset_index(drop=True)
seed_points_snapped = seed_points_snapped[
    ["osmid", "geometry_osm"]
] # drop not-needed columns
# rename geometry column
seed_points_snapped = seed_points_snapped.rename(
    columns={"geometry_osm":"geometry"})

# set "geometry" as geometry column
seed_points_snapped = seed_points_snapped.set_geometry("geometry")
# set osmid as *index* of this df
seed_points_snapped = seed_points_snapped.set_index("osmid")
seed_points_snapped["osmid"] = seed_points_snapped.index

In [ ]:
# plausibility check
seed_points_snapped.plot()

### Greedy triangulation

In [ ]:
# step 1: get list of potential edges, ordered by length
pairs = []
potential_edges = []
distances = []

for pair in combinations(seed_points_snapped["osmid"], 2):

    edge = LineString(seed_points_snapped.loc[list(pair)].geometry)

    pairs.append(pair)
    potential_edges.append(edge)
    distances.append(edge.length)

df = pd.DataFrame(
    {
        "pair": pairs,
        "potential_edge": potential_edges,
        "dist": distances
    }
)

df = df.sort_values(by="dist", ascending=True).reset_index(drop=True)
df = df[df["dist"]>0].reset_index(drop=True) # only keep distances > 0
df.head(10)

In [ ]:
# helper function to check whether newly to be added edge intersects with already added edges
def intersects_properly(geom1, geom2):
    '''
    for 2 shapely geometries, check whether they "properly intersect" (i.e. intersect but not touch, i.e. don't share endpoints)
    '''
    return geom1.intersects(geom2) and not geom1.touches(geom2)

In [ ]:
# step 2: iterate through all potential edges;
# if they dont intersect with existing edges add to multilinestring

current_edges = MultiLineString()
edge_list = []

for i, row in df.iterrows():
    new_edge = row.potential_edge
    pair = row.pair
    if not intersects_properly(current_edges, new_edge):
        current_edges = MultiLineString([linestring for linestring in current_edges.geoms] + [new_edge])
        edge_list.append(pair)

In [ ]:
# step 3: make graph object from edge list
A = nx.Graph()
A.add_nodes_from(seed_points_snapped.index)
A.add_edges_from(edge_list)

In [ ]:
# step 4: add betweenness attributes to edges
bc_values = nx.edge_betweenness_centrality(A, normalized=True)
nx.set_edge_attributes(A, bc_values, name='betweenness_centrality')

In [ ]:
# # commenting this out for now since we don't have ranking by closeness implemented yet
# # step 5: add closeness attributes to nodes
# closeness = nx.closeness_centrality(A)
# nx.set_node_attributes(A, closeness, name='closeness_centrality')

In [ ]:
# step 6: export attributes to gdfs

# as above - commenting out because closeness not yet implemented
# a_nodes = pd.DataFrame.from_dict(
#     nx.get_node_attributes(
#         G=A, 
#         name="closeness_centrality"), 
#     orient="index",
#     columns = ["closeness_centrality"]
# )
# a_nodes["node"] = a_nodes.index
# a_nodes.reset_index(drop=True, inplace=True)

a_edges = pd.DataFrame.from_dict(
    nx.get_edge_attributes(
        G=A,
        name="betweenness_centrality",
    ),
    orient="index",
    columns=["betweenness_centrality"]
)
a_edges["node_tuple"] = a_edges.index
a_edges["source"] = [t[0] for t in a_edges.node_tuple]
a_edges["target"] = [t[1] for t in a_edges.node_tuple]
a_edges.drop(columns=["node_tuple"], inplace=True)

# step 7: rank by attribute/sorting metric
a_edges = a_edges.sort_values(by='betweenness_centrality', ascending = False)
a_edges.reset_index(drop=True, inplace=True)
a_edges["rank"] = a_edges.index # ranking is simply the order of appearance in the betweenness ranking
a_edges.head()

In [ ]:
# step 8: map each abstract edge to a merged geometry of corresponding osmnx edges (routed on g_undir)

# step 8a: get edge list that we can use to index our edges gdf
def get_correct_edgetuples(edge_gdf, nodelist):
    '''
    helper function that maps a node list (output of nx.shortest_paths)
    to the correct set of edge tuples that can be used for INDEXING THE EDGE GDF
    '''
    edgelist_prelim = zip(nodelist, nodelist[1:])
    edgelist_final = []
    for edge_prelim in edgelist_prelim:
        if edge_prelim in edge_gdf.index:
            edgelist_final.append(edge_prelim)
        else:
            edgelist_final.append(tuple([edge_prelim[1], edge_prelim[0]]))
    return edgelist_final

paths = []
for _, row in a_edges.iterrows():
    paths.append(
        nx.shortest_path(
            G=g_undir, # !! use undirected graph here
            source=int(row.source),
            target=int(row.target), 
            weight='length')
    )
a_edges["path_nodes"] = paths
a_edges["path_edges"] = a_edges.path_nodes.apply(lambda x: get_correct_edgetuples(edges, x))
# now the column path_edges contains a set of osmnx edges for each row (abstract edge)

# Step 8b: get "routed" geometry (LineString) for each abstract edge (row)

# get geometry by merging all geoms from edge gdf
a_edges["geometry"] = a_edges.path_edges.apply(
    lambda x: edges.loc[x].geometry.union_all()
)
# convert a_edges into a gdf
a_edges = gpd.GeoDataFrame(a_edges, crs = edges.crs, geometry="geometry")
# merge multilinestring into linestring where possible (should be possible everywhere)
a_edges["geometry"] = a_edges.line_merge()

In [ ]:
# plausibility check: now a_edges contains all of our results we wanted to get,
# will be saving a_edges to file, then plotting.
a_edges.head()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,10))
a_edges.plot(ax=ax, column="rank", cmap="Blues_r")
#seed_points_snapped.plot(ax=ax, color="grey")

In [ ]:
# Step 9: save to file
a_edges.to_file("a_edges.gpkg", driver="GPKG")

### Visualization

In [ ]:
# create directories
os.makedirs("./results/", exist_ok=True)
os.makedirs("./results/plots/", exist_ok=True)
os.makedirs("./results/plots/video/", exist_ok=True)

In [ ]:
# read in file to plot
routed_edges_gdf = gpd.read_file("a_edges.gpkg")

In [ ]:
# viz/plot settings (move to config file later)

# define color palette (from Michael's project: https://github.com/mszell/bikenwgrowth/blob/main/parameters/parameters.py)
streetcolor = "#999999"
edgecolor = "#0EB6D2"
seedcolor = "#ff7338"

# define linewidths

lws = {
   "street": 0.75,
   "bike": 2
}

In [ ]:
for rank in sorted(routed_edges_gdf["rank"].unique()):

    fig, ax = plt.subplots(1,1, figsize=(10,10))

    # first, plot street network as "base line"
    routed_edges_gdf.plot(
        ax=ax,
        color=streetcolor,
        lw=lws["street"],
        zorder=0
    )

    # plot all edges up to current rank

    routed_edges_gdf[routed_edges_gdf["rank"]<=rank].plot(
        ax=ax,
        color=edgecolor,
        lw=lws["bike"],
        zorder=1
    )

    seed_points_snapped.plot(
        ax=ax,
        color=seedcolor,
        zorder=2
    )

    ax.set_axis_off()

    plot_id = "{:03d}".format(rank) # format plot ID with leading zeros
    
    fig.savefig(f"./results/plots/{plot_id}.png", dpi=300)

    plt.close()

In [ ]:
def make_video(
        img_folder_name, # folder where imgs are stored
        fps = 1 # files per second
        ):

    l = glob.glob(f"{img_folder_name}/*.png") # list of filenames

    # assuming the files are called "000.png", "001.png", etc.;
    # to plot them in the right order:
    # remove ".png" from filenames;
    # then, convert filenames to integer
    m = [int(re.findall(r'\d+.png', item)[0].replace(".png", "")) for item in l]
    # and finally, sort:
    images = [l[i] for i in np.argsort(m)]

    # make a "video" subfolder in images folder
    os.makedirs(pathlib.Path(img_folder_name, "video"), exist_ok=True)

    # define file name for video
    video_name = str(pathlib.Path(img_folder_name, "video", "video.mp4"))

    # if there was already such a file - remove it
    if os.path.exists(video_name):
        os.remove(video_name)
        print("\t previous video removed")

    # generate frame in cv2
    frame = cv2.imread(images[0])
    height, width, _ = frame.shape
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video = cv2.VideoWriter(
        video_name,
        fourcc,
        fps, # frames per second = this is the speed of our video
        (width,height)
    )

    # add images as separate frames
    for image in images:
        video.write(
            cv2.resize(
                cv2.imread(image),
                (width, height)
                ),
        )
    cv2.destroyAllWindows()

    # save
    video.release()

    return None

In [ ]:
make_video(
    img_folder_name="./results/plots/",
    fps = 1
)